In [ ]:
# EDA Overview: Mitsui Commodity Prediction

# This notebook loads training data, joins on date_id, checks missingness and target coverage,
# and verifies target_pairs consistency with basic sanity checks.

import os
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path('/Users/fw1230/Documents/Projects/mitsui-commodity-prediction')
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'

TRAIN_FEATURES_PATH = DATA_RAW / 'train.csv'
TRAIN_LABELS_PATH = DATA_RAW / 'train_labels.csv'
TEST_PATH = DATA_RAW / 'test.csv'
TARGET_PAIRS_PATH = DATA_RAW / 'target_pairs.csv'

print(TRAIN_FEATURES_PATH)
print(TRAIN_LABELS_PATH)
print(TEST_PATH)
print(TARGET_PAIRS_PATH)

assert TRAIN_FEATURES_PATH.exists(), 'train.csv missing'
assert TRAIN_LABELS_PATH.exists(), 'train_labels.csv missing'
assert TEST_PATH.exists(), 'test.csv missing'
assert TARGET_PAIRS_PATH.exists(), 'target_pairs.csv missing'


In [ ]:
# Load data
pd.set_option('display.max_columns', 200)

features = pd.read_csv(TRAIN_FEATURES_PATH, low_memory=False)
labels = pd.read_csv(TRAIN_LABELS_PATH, low_memory=False)
test = pd.read_csv(TEST_PATH, low_memory=False)
target_pairs = pd.read_csv(TARGET_PAIRS_PATH)

print('features shape:', features.shape)
print('labels shape:', labels.shape)
print('test shape:', test.shape)
print('target_pairs shape:', target_pairs.shape)

print('features date_id range:', int(features['date_id'].min()), '->', int(features['date_id'].max()))
print('labels date_id range:', int(labels['date_id'].min()), '->', int(labels['date_id'].max()))
print('test date_id range   :', int(test['date_id'].min()), '->', int(test['date_id'].max()))

features.head(3)


In [ ]:
# Join features and labels on date_id
merged = features.merge(labels, on='date_id', how='left', validate='one_to_one')
print('merged shape:', merged.shape)

# Identify feature and target columns
feature_cols = [c for c in features.columns if c != 'date_id']
target_cols = [c for c in labels.columns if c != 'date_id']

print('num features:', len(feature_cols))
print('num targets :', len(target_cols))

merged.head(3)


In [ ]:
# Missingness summary
missing_feat_pct = features[feature_cols].isna().mean().sort_values(ascending=False)
missing_tgt_pct = labels[target_cols].isna().mean().sort_values(ascending=False)

print('Top 20 missing features:')
display(missing_feat_pct.head(20))
print('\nTop 20 missing targets:')
display(missing_tgt_pct.head(20))

# Coverage of targets (non-null counts)
coverage = labels[target_cols].notna().sum().sort_values(ascending=False)
print('\nTarget coverage (non-null counts) - top 20:')
display(coverage.head(20))


In [ ]:
# Corrected target_pairs consistency check - forward-looking log returns
pairs = target_pairs.copy()
pairs = pairs[pairs['target'].str.startswith('target_')].reset_index(drop=True)

# Parse pair string
pairs['symbol_a'] = pairs['pair'].str.split('-').str[0].str.strip()
pairs['symbol_b'] = pairs['pair'].where(~pairs['pair'].str.contains('-'), 
                                       pairs['pair'].str.split('-').str[1]).str.strip()

# Sample for faster computation
sample_pairs = pairs.sample(min(15, len(pairs)), random_state=42)

results = []
for _, row in sample_pairs.iterrows():
    tcol = row['target']
    lag = int(row['lag']) if not pd.isna(row['lag']) else 1
    a = row['symbol_a']
    b = row['symbol_b'] if isinstance(row['symbol_b'], str) else None
    
    if a not in features.columns:
        continue
        
    # Compute the target according to the actual generation code
    tmp = features[['date_id', a]].copy()
    
    if b and b in features.columns:
        # Case 2: Difference of log returns for pairs
        # log(price_A_{t+lag+1} / price_A_{t+1}) - log(price_B_{t+lag+1} / price_B_{t+1})
        tmp['a_return'] = np.log(features[a].shift(-(lag+1)) / features[a].shift(-1))
        tmp['b_return'] = np.log(features[b].shift(-(lag+1)) / features[b].shift(-1))
        tmp['computed_target'] = tmp['a_return'] - tmp['b_return']
    else:
        # Case 1: Log return for single instrument
        # log(price_{t+lag+1} / price_{t+1})
        tmp['computed_target'] = np.log(features[a].shift(-(lag+1)) / features[a].shift(-1))
    
    # Merge with actual target
    tmp = tmp.merge(labels[['date_id', tcol]], on='date_id', how='inner')
    tmp = tmp.sort_values('date_id')
    
    # Compute correlation
    corr = tmp[['computed_target', tcol]].dropna().corr().iloc[0,1]
    n_samples = tmp[['computed_target', tcol]].dropna().shape[0]
    
    results.append({
        'target': tcol, 
        'lag': lag, 
        'symbol_a': a, 
        'symbol_b': b, 
        'type': 'pair_returns' if b else 'single_return',
        'n': n_samples, 
        'corr': corr
    })

pd.DataFrame(results).sort_values('corr', ascending=False)

In [ ]:
# Target distributions and correlations
import matplotlib.pyplot as plt
import seaborn as sns

# Plot distribution of a few sample targets
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
sample_targets = ['target_0', 'target_1', 'target_2', 'target_3']

for i, target in enumerate(sample_targets):
    if target in merged.columns:
        row, col = i // 2, i % 2
        merged[target].dropna().hist(bins=50, ax=axes[row, col], alpha=0.7)
        axes[row, col].set_title(f'{target} Distribution')
        axes[row, col].set_xlabel('Value')
        axes[row, col].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Target correlation heatmap (first 20 targets)
target_subset = [col for col in target_cols[:20] if col in merged.columns]
corr_matrix = merged[target_subset].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=False, cmap='RdBu_r', center=0)
plt.title('Target Correlations (First 20)')
plt.show()

In [ ]:
# Group features by type and get summary stats
feature_groups = {
    'JPX_Futures': [col for col in feature_cols if 'JPX_' in col],
    'LME_Metals': [col for col in feature_cols if 'LME_' in col],
    'US_Stocks': [col for col in feature_cols if 'US_Stock_' in col],
    'FX_Pairs': [col for col in feature_cols if 'FX_' in col]
}

print("Feature Group Sizes:")
for group_name, group_cols in feature_groups.items():
    print(f"{group_name}: {len(group_cols)} features")

# Summary stats for each group
for group_name, group_cols in feature_groups.items():
    if group_cols:
        print(f"\n{group_name} Summary Stats:")
        print(merged[group_cols].describe())

for group_name, group_cols in feature_groups.items():
    if len(group_cols) > 1:
        print(f"\n{group_name} Correlation Matrix (first 10 features):")
        subset_cols = group_cols[:10]  # Limit to avoid huge matrices
        corr_matrix = merged[subset_cols].corr()
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, fmt='.2f')
        plt.title(f'{group_name} Feature Correlations')
        plt.xticks(rotation=45)
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()

In [ ]:
# Plot key features over time
key_features = ['LME_AH_Close', 'JPX_Gold_Standard_Futures_Close', 'US_Stock_VT_adj_close']

fig, axes = plt.subplots(len(key_features), 1, figsize=(12, 10))
for i, feature in enumerate(key_features):
    if feature in merged.columns:
        merged.plot(x='date_id', y=feature, ax=axes[i], legend=False)
        axes[i].set_title(f'{feature} Over Time')
        axes[i].set_ylabel('Value')

plt.tight_layout()
plt.show()

# Volatility analysis (rolling std) - CORRECTED
fig, axes = plt.subplots(len(key_features), 1, figsize=(12, 10))
for i, feature in enumerate(key_features):
    if feature in merged.columns:
        rolling_std = merged[feature].rolling(5).std()
        axes[i].plot(merged['date_id'], rolling_std)
        axes[i].set_title(f'{feature} Rolling 20-period Std Dev')
        axes[i].set_ylabel('Volatility')

plt.tight_layout()
plt.show()

In [ ]:
# Missing value patterns by group
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
for i, (group_name, group_cols) in enumerate(feature_groups.items()):
    if group_cols:
        row, col = i // 2, i % 2
        missing_pct = merged[group_cols].isna().mean().sort_values(ascending=False)
        missing_pct.head(10).plot(kind='bar', ax=axes[row, col])
        axes[row, col].set_title(f'{group_name} Missing Values (%)')
        axes[row, col].set_ylabel('Missing %')
        axes[row, col].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Identify price columns (exclude date_id and any non-price columns)
price_cols = [col for col in feature_cols if not col.endswith('_missing') and col != 'date_id']

print(f"Creating log returns for {len(price_cols)} price columns")

# Create log returns
log_returns = np.log(features[price_cols] / features[price_cols].shift(1))

# Rename columns to indicate they're returns
log_returns.columns = [f"{col}_log_return" for col in price_cols]

# Check missing values in log returns
missing_log_returns = log_returns.isna().sum().sort_values(ascending=False)
print("\nMissing values in log returns (top 20):")
print(missing_log_returns.head(20))

# Forward fill missing values (but leave high-missing features as-is)
log_returns_filled = log_returns.copy()

# Only forward fill features with reasonable missing rates (<50%)
reasonable_missing = missing_log_returns[missing_log_returns <= len(log_returns) * 0.5]
high_missing = missing_log_returns[missing_log_returns > len(log_returns) * 0.5]

print(f"\nFeatures with reasonable missing rates (<50%): {len(reasonable_missing)}")
print(f"Features with high missing rates (>50%): {len(high_missing)}")

# Forward fill only the reasonable ones
log_returns_filled[reasonable_missing.index] = log_returns_filled[reasonable_missing.index].fillna(method='ffill')

# Combine original features with log returns
features_with_returns = pd.concat([
    features.reset_index(drop=True),
    log_returns_filled.reset_index(drop=True)
], axis=1)

print(f"\nFinal features shape: {features_with_returns.shape}")
print(f"Original features: {len(feature_cols)}")
print(f"New log return features: {len(log_returns.columns)}")

# Show some examples of the new features
print("\nSample of new log return features:")
print(features_with_returns[['date_id'] + list(log_returns.columns[:5])].head())